## Preparation

In [179]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

In [143]:
df = pd.DataFrame(columns=[
	'company',
	'company_size',
	'job_title',
	'level',
	'domain',
	'yoe_total',
	'yoe_at_company',
	'total_compensation',
	'location'
])

## Site: levels.fyi

In [183]:
page = requests.get('https://www.levels.fyi/companies')
soup = BeautifulSoup(page.content, 'html.parser')
companies_container = soup.find_all(
    'h6', class_='MuiTypography-root MuiTypography-h6 css-1v6gvkr')
companies = [company.text for company in companies_container]
companies

['Google',
 'Amazon',
 'Apple',
 'Lyft',
 'Facebook',
 'Microsoft',
 'Uber',
 'Stripe',
 'Roblox',
 'Coinbase',
 'Databricks',
 'Snap',
 'Netflix',
 'LinkedIn',
 'Salesforce',
 'Hudson River Trading',
 'Jane Street',
 'Citadel',
 'Two Sigma',
 'JPMorgan Chase',
 'Capital One',
 'Oracle',
 'Bytedance',
 'Intel']

In [184]:
def crawl_company(company_id):
	page = requests.get(f'https://www.levels.fyi/companies/{company_id}')
	soup = BeautifulSoup(page.content, 'html.parser')
	size = soup.findAll(
		'h6', class_='MuiTypography-root MuiTypography-subtitle1 css-idrr7q')[1].text

	page = requests.get(f'https://www.levels.fyi/companies/{company_id}/salaries')
	soup = BeautifulSoup(page.content, 'html.parser')
	job_titles_container = soup.find_all('h6', class_='MuiTypography-root MuiTypography-h6 css-jv9qtm')
	job_titles = [job_title.text for job_title in job_titles_container]
	job_title_ids = [job_title.strip().lower().replace(' ', '-') for job_title in job_titles]

	driver = webdriver.Chrome()

	for job_title_id in job_title_ids:
		driver.get(f'https://www.levels.fyi/company/{company_id}/salaries/{job_title_id}')

		print(f'crawling {company_id} {job_title_id}...')

		try:
			button = driver.find_element(
				by=By.CSS_SELECTOR, value="button.MuiButton-root.MuiButton-text.MuiButton-textPrimary.MuiButton-sizeMedium.MuiButton-textSizeMedium.MuiButtonBase-root.css-um5318")
			button.click()

			button = driver.find_element(
				by=By.CSS_SELECTOR, value="button.MuiButton-root.MuiButton-text.MuiButton-textPrimary.MuiButton-sizeMedium.MuiButton-textSizeMedium.MuiButtonBase-root.css-g9gvkf")
			button.click()
		except:
			pass

		try:
			tbody = driver.find_element(
				by=By.CSS_SELECTOR, value="tbody.MuiTableBody-root.css-6l3v2e"
			)
			rows = tbody.find_elements(
				by=By.TAG_NAME, value="tr"
			)
			for row in rows:
				try:
					cells = row.find_elements(
						by=By.TAG_NAME, value="td"
					)
					location = cells[0].find_element(
						by=By.TAG_NAME, value="span"
					).find_element(
						by=By.TAG_NAME, value="span"
					).text.split('|')[0]
					level = cells[1].find_element(
						by=By.TAG_NAME, value="p"
					).text
					field = cells[1].find_element(
						by=By.TAG_NAME, value="span"
					).text
					yoe_total = cells[2].find_element(
						by=By.TAG_NAME, value="p"
					).text
					yoe_at_company = cells[2].find_element(
						by=By.TAG_NAME, value="span"
					).text
					total_compensation = cells[3].find_element(
						by=By.TAG_NAME, value="p"
					).text
					df = df.append({
						'company': company_id.replace('-', ' ').title(),
						'company_size': size,
						'job_title': job_title_id.replace('-', ' ').title(),
						'level': level,
						'domain': field,
						'yoe_total': yoe_total,
						'yoe_at_company': yoe_at_company,
						'total_compensation': total_compensation,
						'location': location
					}, ignore_index=True)
				except:
					continue
		except:
			pass
		
	driver.close()

for company in companies:
	crawl_company(company)

df


crawling Google software-engineer...
crawling Google product-manager...
crawling Google product-designer...
crawling Google technical-program-manager...
crawling Google sales...
crawling Google software-engineering-manager...
crawling Google data-scientist...
crawling Google program-manager...
crawling Google hardware-engineer...
crawling Google marketing...
crawling Google project-manager...
crawling Google solution-architect...
crawling Google recruiter...
crawling Google financial-analyst...
crawling Google accountant...
crawling Google business-analyst...
crawling Google human-resources...
crawling Google mechanical-engineer...
crawling Google business-development...
crawling Google marketing-operations...
crawling Google information-technologist...
crawling Google management-consultant...
crawling Google sales-engineer...
crawling Google security-analyst...
crawling Google technical-writer...
crawling Google legal...
crawling Google product-design-manager...
crawling Google data-s

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"tbody.MuiTableBody-root.css-6l3v2e"}
  (Session info: chrome=119.0.6045.160); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6262F82B2+55298]
	(No symbol) [0x00007FF626265E02]
	(No symbol) [0x00007FF6261205AB]
	(No symbol) [0x00007FF62616175C]
	(No symbol) [0x00007FF6261618DC]
	(No symbol) [0x00007FF62619CBC7]
	(No symbol) [0x00007FF6261820EF]
	(No symbol) [0x00007FF62619AAA4]
	(No symbol) [0x00007FF626181E83]
	(No symbol) [0x00007FF62615670A]
	(No symbol) [0x00007FF626157964]
	GetHandleVerifier [0x00007FF626670AAB+3694587]
	GetHandleVerifier [0x00007FF6266C728E+4048862]
	GetHandleVerifier [0x00007FF6266BF173+4015811]
	GetHandleVerifier [0x00007FF6263947D6+695590]
	(No symbol) [0x00007FF626270CE8]
	(No symbol) [0x00007FF62626CF34]
	(No sy